In [1]:
from transformers import MimiModel, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")
model = MimiModel.from_pretrained("kyutai/mimi")
model = model.to("cuda")


/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/transformers/models/mimi/modeling_mimi.py:163: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [16]:
import numpy as np
import torch
import torchaudio

test_arr = np.load("../../fish-speech.rs/out.npy")
test_input = torch.from_numpy(test_arr[:,:200]).to("cuda").to(torch.long).unsqueeze(0)

# test_input = torch.randint(low=0, high=2048, size=(8, 175)).unsqueeze(0).to("cuda")
out_pcm = model.decode(test_input)
torchaudio.save("out.wav", out_pcm.audio_values[0].to("cpu"), 24000)